In [ ]:
# %conda install --file requirements-poetry.txt
%conda install toml pyarrow jupysql -c conda-forge

: 

In [ ]:
# %pip install 

In [12]:
# download assets from block storage at given URL 
# (our case is zipped folder served through LAN from another machine by miniserve)
%reload_ext dotenv
# %dotenv
import os
from urllib.request import urlretrieve
from pathlib import Path

ASSET_URL, out_file= (os.environ('ASSET_URL') ,'out/testing_data.tar.gz')

if not Path(out_file).is_file():
    os.makedirs(os.path.dirname(out_file), exist_ok=True)
    folder_out, res = urlretrieve(ASSET_URL, filename=out_file)
    print(folder_out)

In [2]:
import tarfile

# Extract files from tar
with tarfile.open(out_file, "r") as file:
    for each in file.getnames():
        print(each)
        file.extract(f'{each}')

data
data/en.openfoodfacts.org.products.csv
data/en.openfoodfacts.org.products-delimited.csv


In [3]:
import polars as pl

df = pl.scan_csv("./data/en.openfoodfacts.org.products-delimited.csv",
                 truncate_ragged_lines=True,
                 )
df.sink_parquet('./data/openfoodfacts_products-delimited.parquet',
                compression="snappy",
                row_group_size=100_000,
                statistics=True,
                )

In [3]:
%reload_ext pyprql.magic
%reload_ext sql

%config PrqlMagic.autopolars = True
%config SqlMagic.autopolars = True
# recipes = os.environ['RECIPES_ASSET_URL']
# !wget http://192.168.0.143:3003/recipes.parquet
# !curl -sL https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv -o penguins.csv

%prql duckdb:///doto.duckdb
%sql duckdb:///doto.duckdb

# %sql CREATE TABLE off_products AS SELECT * FROM read_parquet('./data/openfoodfacts_products.parquet') LIMIT 200
# df = %sql SELECT * FROM read_parquet('./data/openfoodfacts_products.parquet')


# df = %sql SELECT * FROM test_recipes WHERE REGEXP_MATCHES(name, 'mex*') 
# %sql SELECT n_ingredients FROM read_parquet('http://192.168.0.143:3003/recipes.parquet') WHERE REGEXP_MATCHES(name, 'mex*')
# %sqlplot pie --table test_recipes --column minutes
# TODO: doesn't work w/ polars
# %sqlplot histogram --table test_recipes --column n_ingredients --bins 100

# %sql SELECT * FROM test_recipes WHERE REGEXP_MATCHES(name, 'mex*')
%%sql 
SELECT * FROM test_recipes 
WHERE REGEXP_MATCHES(name, 'mex*')
# display(df)

# df = %prql prql target:sql.duckdb
# prql target:sql.duckdb
# from `test_recipes`  
# sort {-n_ingredients} 
# display(df)


SyntaxError: invalid syntax (1085949981.py, line 25)